# Elastic Search Playground

In [1]:
ES_API = 'https://localhost:9200'
CERT = '../http_ca.crt'
USER = 'elastic'
PW = ''

Check if the Elastic Search instance is running

In [4]:
import requests
import json

requests.get(ES_API, auth=(USER, PW), verify=CERT)

<Response [200]>

What is the overall status of the elastic search instance?

In [5]:
res = requests.get(ES_API+'/_cluster/health', auth=(USER, PW), verify=CERT) 
json.loads(res.text)

{'cluster_name': 'docker-cluster',
 'status': 'green',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 11,
 'active_shards': 11,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 0,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 100.0}

---
## Create Index for Tweets

In [6]:
res = requests.put(ES_API+'/tweets', auth=(USER, PW), verify=CERT)
json.loads(res.text)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'tweets'}

---
## Feed data into ES

In [7]:
data = {"name": "Ute Braun"}

res = requests.post(ES_API+'/tweets/_doc', auth=(USER, PW), verify=CERT, json=data)
json.loads(res.text)

{'_index': 'tweets',
 '_id': 'CwlO6IQBSkByIYEnGchC',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 1}

---
## Search data

In [8]:
res = requests.get(ES_API+'/tweets/_search', auth=(USER, PW), verify=CERT)

res_json = json.loads(res.text)
res_json["hits"]["hits"]

[{'_index': 'tweets',
  '_id': 'CwlO6IQBSkByIYEnGchC',
  '_score': 1.0,
  '_source': {'name': 'Ute Braun'}}]

---
## Delete Index

In [3]:
res = requests.delete(ES_API+'/tweets/', auth=(USER, PW), verify=CERT)
res.text

'{"acknowledged":true}'